In [1]:
# pandas 라이브러리 가져오기
import pandas as pd

### 중복값 확인

- 레코드(행) 단위로 동일한 값이 저장된 데이터(레코드)

In [4]:
# 5행 2열 데이터프레임
# 모든 데이터가 범주형인 값으로 사용
# 동일한 레코드가 있도록 중복 적용
df = pd.DataFrame({'key1':['a','b','c','b','b'],
                   'key2':['x','x','y','z','z']})

df

,key1,key2
0,a,x
1,b,x
2,c,y
3,b,z
4,b,z


In [5]:
# 로우 기준으로 중복값 확인 
# df.duplicated(keep='first')
df.duplicated()

0    False
1    False
2    False
3    False
4     True
dtype: bool

In [8]:
# 모든 중복값에 대해 True
# keep = False
df.duplicated(keep=False)

0    False
1    False
2    False
3     True
4     True
dtype: bool

In [9]:
# 1,2,3,4,5 를 데이터로 가지는 컬럼 추가된 데이터프레임
df['key3'] = [1,2,3,4,5]
df

,key1,key2,key3
0,a,x,1
1,b,x,2
2,c,y,3
3,b,z,4
4,b,z,5


In [10]:
# 전체 컬럼을 포함한 레코드에 대한 중복값 확인
df.duplicated(keep=False)

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [11]:
# 특정 컬럼들 기준으로 중복값 확인 1
df[['key1', 'key2']].duplicated()

0    False
1    False
2    False
3    False
4     True
dtype: bool

In [12]:
# 특정 컬럼들 기준으로 중복값 확인 2
df.duplicated(['key1', 'key2'])

0    False
1    False
2    False
3    False
4     True
dtype: bool

In [15]:
df['key1']

0    a
1    b
2    c
3    b
4    b
Name: key1, dtype: object

In [19]:
# 하나의 컬럼 기준으로 중복값 확인하기 1
df['key1'][df['key1'].duplicated(keep=False)]

1    b
3    b
4    b
Name: key1, dtype: object

In [20]:
df['key1'].value_counts()

b    3
a    1
c    1
Name: key1, dtype: int64

In [21]:
# 하나의 컬럼 기준으로 중복값 확인하기 2
# keep = 'first'(default) / 'last' / False
df.duplicated('key1', keep=False)

0    False
1     True
2    False
3     True
4     True
dtype: bool

### 중복값 삭제

- df.drop_duplicates()

In [22]:
df

,key1,key2,key3
0,a,x,1
1,b,x,2
2,c,y,3
3,b,z,4
4,b,z,5


In [23]:
# keep='first'
# key1, key2 컬럼에 대한 중복값 삭제하기 
df[['key1', 'key2']].drop_duplicates()

,key1,key2
0,a,x
1,b,x
2,c,y
3,b,z


In [26]:
# key1 컬럼의 중복값 삭제
# 기본동작 : 첫 중복값을 유지하고 나머지는 삭제
df['key1'].drop_duplicates()

0    a
1    b
2    c
Name: key1, dtype: object

In [28]:
# 모든 중복 삭제
# 중복이 없는 데이터 확인
df['key1'].drop_duplicates(keep=False)

0    a
2    c
Name: key1, dtype: object

### 결측값 형식통일/치환

In [29]:
# 누락데이터가 있는 숫자 값 데이터 생성
# 결측치를 숫자로 표현 : -9999, -100000
s1 = pd.Series([1,2,-9999,3,4,-9999,5,-9999])
s1

0       1
1       2
2   -9999
3       3
4       4
5   -9999
6       5
7   -9999
dtype: int64

In [30]:
s1.isnull().sum()

0

In [32]:
# 결측치 값 생성
import numpy as np

np.nan

nan

In [34]:
# pandas에서 인식하는 결측치로 변형
# -9999 => np.nan
s1.replace(-9999, np.nan, inplace=True)

In [35]:
s1.isnull().sum()

3

In [36]:
s2 = pd.Series([1,2,-999,3,4,-9999,5,-100000])

In [39]:
# 여러 개의 값에 대해 결측치로 변형 1 : 리스트
s2.replace([-999, -9999, -100000], np.nan, inplace=True)

In [40]:
s2.isnull().sum()

3

In [ ]:
# 여러 개의 값에 대해 결측치로 변형 2 : 딕셔너리
# s2.replace({-999:np.nan, -100000:np.nan}, inplace=True)

### 범주형 데이터

#### 1. 연속형 데이터를 범주형 데이터로 변경

- 예시) 나이 : 15, 22, 67, 45, 55 ... -> 나이대/연령대(유도변수) : 10대, 20대, 30대 ...
- pd.cut(data, bins|scalar, labels)
    - 주어진 구간 또는 개수에 따라 그룹 생성
    - 개수는 구간을 균등한 길이로 나눔
- pd.qcut(data, bins|scalar)
    - 주어진 구간 또는 개수에 따라 그룹 생성
    - 구간은 0 이상 1 미만
    - 개수는 적당한 크기의 그룹을 생성하는 길이로 나눔

In [43]:
# 샘플 데이터 생성
sample = np.random.randint(20,size=20)
sample

array([17,  4, 11,  8,  6,  6,  3, 11,  3, 14,  6,  3, 18, 11,  6,  5,  5,
        7, 16,  9])

In [44]:
# pd.cut()
# 구간 : bins=[0,5,10,15,20]
# 1) 구간을 지정해서 범주 생성
# 0초과 5이하 : (0,5] 
# 5초과 10이하 : (5,10]
# 10초과 15이하 : (10,15]
# 15초과 20이하 : (15,20]
bins=[0,5,10,15,20]

In [46]:
# cut() 반환값(속성) : codes, categories
result1 = pd.cut(sample, bins)

In [47]:
# 분류된 범주 : codes
result1.codes

array([3, 0, 2, 1, 1, 1, 0, 2, 0, 2, 1, 0, 3, 2, 1, 0, 0, 1, 3, 1],
      dtype=int8)

In [48]:
# 분류된 범위/구간
result1.categories

IntervalIndex([(0, 5], (5, 10], (10, 15], (15, 20]],
              closed='right',
              dtype='interval[int64]')

In [54]:
# 범주별 개수
result1.value_counts()
pd.value_counts(result1)

(5, 10]     7
(0, 5]      6
(10, 15]    4
(15, 20]    3
dtype: int64

In [55]:
# 분류 범주명 설정
# labels 
# 범주명 개수 = len(bins) -1
result2 = pd.cut(sample, bins, labels=['A','B','C','D'])
result2.codes

array([3, 0, 2, 1, 1, 1, 0, 2, 0, 2, 1, 0, 3, 2, 1, 0, 0, 1, 3, 1],
      dtype=int8)

In [56]:
result2.categories

Index(['A', 'B', 'C', 'D'], dtype='object')

In [57]:
result2.value_counts()

A    6
B    7
C    4
D    3
dtype: int64

In [58]:
# 최종 분류하려는 범주의 개수가 정해져 있고
# 범주 간의 간격이 동일한 경우
result3 = pd.cut(sample, 4)

In [59]:
result3.codes

array([3, 0, 2, 1, 0, 0, 0, 2, 0, 2, 0, 0, 3, 2, 0, 0, 0, 1, 3, 1],
      dtype=int8)

In [60]:
result3.value_counts()

(2.985, 6.75]    10
(6.75, 10.5]      3
(10.5, 14.25]     4
(14.25, 18.0]     3
dtype: int64

In [61]:
# pd.qcut()
result4 = pd.qcut(sample, 4)

In [62]:
# 분류된 범위/구간 
result4.categories

IntervalIndex([(2.999, 5.0], (5.0, 6.5], (6.5, 11.0], (11.0, 18.0]],
              closed='right',
              dtype='interval[float64]')

In [63]:
result4.value_counts()

(2.999, 5.0]    6
(5.0, 6.5]      4
(6.5, 11.0]     6
(11.0, 18.0]    4
dtype: int64

#### 2. sklearn 라이브러리를 이용한 범주형 데이터 생성

- LabelEncoder() : 문자형 범주를 숫자형 변주로 변환
    - fit_transform() : 범주형 데이터로 변환해서 반환
    - inverse_transform() : 변환된 결과를 원래 데이터로 복구
    
- OneHotEncoder() : 범주형 데이터를 원핫인코딩을 적용하여 모델 입력값으로 사용할 수 있게 변환
    - fit_transform()
    - inverse_transform()

In [64]:
# 라이브러리 가져오기
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [65]:
# 샘플 데이터프레임 생성
# 3x4
# 컬럼 : color, size1, size2, label_name
data = [['green', 'M', '10', 'label1'],
        ['red', 'XL', '30', 'label2'],
        ['black', 'L', '20', 'label3']]

df = pd.DataFrame(data, columns=['color', 'size1', 'size2', 'label_name'])
df

,color,size1,size2,label_name
0,green,M,10,label1
1,red,XL,30,label2
2,black,L,20,label3


In [66]:
# LabelEncoder 객체 생성
encoder = LabelEncoder()

In [67]:
# fit()
encoder.fit(df['color'])

LabelEncoder()

In [69]:
# fit() 결과
encoder.classes_

array(['black', 'green', 'red'], dtype=object)

In [71]:
# fit() 실행한 객체에 대해 변환된 결과를 반환
encoder.transform(df['color'])

array([1, 2, 0])

In [73]:
# fit_transform()
y = encoder.fit_transform(df['size1'])

In [74]:
# 원본 데이터로 복구
encoder.inverse_transform(y)

array(['M', 'XL', 'L'], dtype=object)

In [83]:
data = [['green', 'M', '10', 'label1'],
        ['red', 'XL', '30', 'label2'],
        ['black', 'L', '20', 'label3']]

df2 = pd.DataFrame(data, columns=['color', 'size1', 'size2', 'label_name'])
df2

,color,size1,size2,label_name
0,green,M,10,label1
1,red,XL,30,label2
2,black,L,20,label3


In [75]:
# OneHotEncoder 적용
OHE = OneHotEncoder()

In [87]:
x1 = encoder.fit_transform(df2['size1'])

In [88]:
# fit_transform()
tmp = OHE.fit_transform(x1.reshape(-1,1)).toarray()

In [89]:
pd.DataFrame(tmp)

,0,1,2
0,0.0,1.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
